In [ ]:
import matplotlib.pyplot as plt
import imageio, os 
import skimage.transform
import numpy as np

dir_image = "image"
img = imageio.imread(os.path.join(dir_image,"person.JPG"))
newsize = (200,250)
img = skimage.transform.resize(img,newsize)
const = 4
plt.figure(figsize=(5*const,6*const))
plt.imshow(img)
plt.show()

In [ ]:
import selective_search as ss
regions = ss.get_region_proposal(img,min_size=50)

In [ ]:
import seaborn as sns
def plt_rectangle(plt,label,x1,y1,x2,y2,color = "yellow", alpha=0.5):
    linewidth = 3
    if type(label) == list:
        linewidth = len(label)*3 + 2
        label = ""
        
    plt.text(x1,y1,label,fontsize=20,backgroundcolor=color,alpha=alpha)
    plt.plot([x1,x1],[y1,y2], linewidth=linewidth,color=color, alpha=alpha)
    plt.plot([x2,x2],[y1,y2], linewidth=linewidth,color=color, alpha=alpha)
    plt.plot([x1,x2],[y1,y1], linewidth=linewidth,color=color, alpha=alpha)
    plt.plot([x1,x2],[y2,y2], linewidth=linewidth,color=color, alpha=alpha)
    
    
plt.figure(figsize=(20,20))    
plt.imshow(img)
for item, color in zip(regions,sns.xkcd_rgb.values()):
    x1, y1, width, height = item["rect"]
    label = item["labels"][:5]
    plt_rectangle(plt,label,
                  x1,
                  y1,
                  x2 = x1 + width,
                  y2 = y1 + height, 
                  color= color)

In [ ]:
import numpy as np 

def warp_candidate_regions(img,regions):
    newsize_cnn = (224, 224)
    X = []
    for i, r in enumerate(regions):
        origx , origy , width, height = r["rect"]
        candidate_region = img[origy:origy + height,
                               origx:origx + width]
        img_resize = skimage.transform.resize(candidate_region,newsize_cnn)
        X.append(img_resize)

    X = np.array(X)
    print(X.shape)
    return(X)
X = warp_candidate_regions(img,regions)

In [ ]:
from keras.applications import VGG16
modelvgg16 = VGG16(include_top=True,weights='imagenet')
modelvgg16.summary()

In [ ]:
from keras import models
modelvgg = models.Model(inputs  = modelvgg16.inputs, 
                        outputs = modelvgg16.layers[-3].output)

modelvgg.summary()

In [ ]:
import time
start   = time.time()
feature = modelvgg.predict(X)
end     = time.time()
print("TIME TOOK: {:5.4f}MIN".format((end-start)/60.0))
feature.shape

In [ ]:
from keras.models import load_model

dir_result = "output"
classifier = load_model(os.path.join(dir_result,"classifier.h5"))
classifier.summary()
y_pred = classifier.predict(feature)

In [ ]:
def plot_selected_regions_with_estimated_prob(y_pred,
                                              method="highest",
                                              upto=1):
    irows = np.argsort(y_pred[:,0])
    if method == "highest":
        irows = irows[::-1]
    count = 1
    const = 4
    fig = plt.figure(figsize=(upto*const,const))
    fig.subplots_adjust(hspace=0.13,wspace=0.0001,
                        left=0,right=1,bottom=0, top=1)
    for irow in irows:
        prob = y_pred[irow,0]
        r    = regions[irow]
        origx , origy , width, height = r["rect"]
        
        ax = fig.add_subplot(1,upto,count)
        ax.imshow(img)
        ax.axis("off")
        plt_rectangle(ax,label="",
                      x1=origx,
                      y1=origy,
                      x2=origx + width,
                      y2=origy+height,color = "yellow", alpha=0.5)
        ax.set_title("Prob={:4.3f}".format(prob))
        count += 1
        if count > upto:
            break
    plt.show()
print("The most likely candidate regions")    
plot_selected_regions_with_estimated_prob(y_pred,method="highest",upto=1)